# IFSP - Câmpus Campinas

## Análise sobre a previsibilidade das decisões de admissibilidade no Superior Tribunal de Justiça usando inteligência artificial

#### Aluno: Guilherme Cioldin Dainese

### Parte 1 - Carregamento dos dados abertos do Superior Tribunal de Justiça

O Superior Tribunal de Justiça (STJ) utiliza um portal para disponibilizar dados do Tribunal para acesso público. O portal está disponível no endereço https://dadosabertos.web.stj.jus.br/.

O dataset específico para esse trabalho é o "Íntegras de Decisões Terminativas e Acórdãos do Diário da Justiça", com o link específico: https://dadosabertos.web.stj.jus.br/dataset/integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica.

Esse dataset possui as decisões monocráticas indicadas pelos gabinetes de ministros como terminativas e que tenham sido proferidas diretamente no processo principal.

O dataset possui três tipos de arquivos. Primeiro, um arquivo .csv que é utilizado unicamente como dicionário das entradas nos arquivos JSON. Os arquivos JSON por sua vez possuem os metadados das decisões e por fim, cada entrada possui um arquivo .zip com o texto das decisões em si.

Os arquivos .zip serão ignorados pois os dados serão processados diretamente das petições constantes dos processos judiciais.

Todavia, será feito o download dos arquivos .json para construir a base de dados inicial que permitirá a seleção das petições.

In [1]:
from pprint import pprint
from time import sleep
from tqdm.notebook import tqdm
import json
import pandas as pd
import re
import requests

In [2]:
#Endereço da raiz da base de dados do CNJ
api_url_root = 'https://dadosabertos.web.stj.jus.br/'
#Lista de pacotes seguindo o esquema da API CKAN
package_list = 'api/3/action/package_list'

In [3]:
#Exibe a lista de pacotes
pprint(requests.get(api_url_root + package_list).json())

{'help': 'https://dadosabertos.web.stj.jus.br/api/3/action/help_show?name=package_list',
 'result': ['api-publica-datajud',
            'atas-de-distribuicao',
            'espelhos-de-acordaos-corte-especial',
            'espelhos-de-acordaos-primeira-secao',
            'espelhos-de-acordaos-primeira-turma',
            'espelhos-de-acordaos-quarta-turma',
            'espelhos-de-acordaos-quinta-turma',
            'espelhos-de-acordaos-segunda-secao',
            'espelhos-de-acordaos-segunda-turma',
            'espelhos-de-acordaos-sexta-turma',
            'espelhos-de-acordaos-terceira-secao',
            'espelhos-de-acordaos-terceira-turma',
            'integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica',
            'movimentacao-processual',
            'plano-estrategico-stj',
            'precedentes-qualificados'],
 'success': True}


O pacote está identificado como `integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica`

In [4]:
package_id = 'integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica'

In [5]:
#Carrega dados de pacote da API do STJ
response = requests.get(f'{api_url_root}/api/3/action/package_show?id={package_id}') #Comando padrão da API CKAN para carregar os dados
package_info = json.loads(response.text) #Carrega o arquivo .json
resources = package_info['result']['resources'] #Salva os recursos da query

In [6]:
#Exibe o primeiro resultado
resources[0]

{'cache_last_updated': None,
 'cache_url': None,
 'created': '2022-05-19T20:01:02.643048',
 'datastore_active': False,
 'description': 'Dicionário de dados. ',
 'format': 'CSV',
 'hash': '',
 'id': '0488e8aa-33a3-494d-86ef-17889a0155ee',
 'last_modified': '2022-05-19T20:01:02.627455',
 'metadata_modified': '2022-05-19T20:02:05.698389',
 'mimetype': 'text/csv',
 'mimetype_inner': None,
 'name': 'dicionariointegrasdecisoes.csv',
 'package_id': 'a2cd85cc-1391-4ebc-aeed-a45dd75a7987',
 'position': 0,
 'resource_type': None,
 'size': 1536,
 'state': 'active',
 'url': 'https://dadosabertos.web.stj.jus.br/dataset/a2cd85cc-1391-4ebc-aeed-a45dd75a7987/resource/0488e8aa-33a3-494d-86ef-17889a0155ee/download/dicionariointegrasdecisoes.csv',
 'url_type': 'upload'}

Esse resultado é o arquivo .csv que é o dicionário das decisões do STJ nesse dataset.

In [7]:
dicionario = requests.get(resources[0]['url'])
if dicionario.status_code == 200:
        with open(f"./misc/{resources[0]['name']}", 'xb') as f:
            f.write(dicionario.content.decode('iso-8859-1').encode('utf-8'))
    #Foi necessário converter a codificação do arquivo.

Salvo o arquivo de dicionário, resta fazer o download dos arquivos de metadados.

Considerando a entrada em vigor da emenda constitucional n.º 125 de 2022, serão selecionados apenas os processos com data de decisão no STJ posterior a setembro de 2022.

In [8]:
# Filtrar apenas os arquivos do tipo JSON
arquivos_json = [resource for resource in resources if resource["format"].lower() == "json"]

Os arquivos variam de nome entre metadados\[ano]\[mês]\[dia].json.

Para filtrar as decisões posteriores a essa data utilizaremos expressões regulares.

In [9]:
#Cria expressão regular para excluir as entradas anteriores a setembro de 2022.
#Não há necessidade de controlar os anos na expressão regular pois a primeira entrada é em março de 2022.
pattern = re.compile(r'metadados20220[1-8]')
json_selecionados = [resource for resource in arquivos_json if not pattern.match(resource["name"])]

In [10]:
# Itera sobre todas as entradas selecionadas para baixar os arquivos de metadados
arquivos_falha = []

for resource in tqdm(json_selecionados):
    response = requests.get(resource["url"])
    if response.status_code == 200:
        with open('./json_metadados/' + resource["name"], 'xb') as file:
            file.write(response.content)
    else:
        print(f"Falha ao baixar {resource['name']}, status code: {response.status_code}")
        arquivos_falha.append(resource['name'])

  0%|          | 0/314 [00:00<?, ?it/s]

Falha ao baixar metadados20221027.json, status code: 520
Falha ao baixar metadados20221107.json, status code: 520
Falha ao baixar metadados20221124.json, status code: 520
Falha ao baixar metadados20230130.json, status code: 520
Falha ao baixar metadados20230223.json, status code: 520
Falha ao baixar metadados20230629.json, status code: 520
Falha ao baixar metadados20230706.json, status code: 520
Falha ao baixar metadados20231109.json, status code: 520
Falha ao baixar metadados20231110.json, status code: 520
Falha ao baixar metadados20240110.json, status code: 520


In [11]:
arquivos_falha

['metadados20221123.json',
 'metadados20221205.json',
 'metadados20221215.json',
 'metadados20221216.json',
 'metadados20221219.json',
 'metadados20221220.json']

Realizado o download, resta apenas unir os arquivos em um único dataframe para posterior processamento.

In [ ]:
#Une todos os metadados em um único dataframe
dfs = []
for json in json_selecionados:
    df = pd.read_json(f"./json_metadados/{json['name']}")
    dfs.append(df)
#Retorna um dataframe único com todos os dados baixados
dataframe = pd.concat(dfs, ignore_index=True)

In [ ]:
#Exibe do dataframe concatenado
dataframe

Todos os dados relevantes disponíveis no repositório já foram baixados e unidos em um único dataframe. Salvaremos o arquivo para seguir com o processamento.

A opção de arquivo (feather) é utilizada para permitir maior rapidez de salvamento e menor espaço.

In [ ]:
dataframe.to_feather('./dataframes/json_raw.feather')

In [ ]:
#751914 rows × 12 columns